In [71]:
import pandas as pd
import requests
import json
import datetime

In [72]:
securities = ['GAZP', 'SBER', 'GMKN', 'LKOH', 'ROSN']

In [63]:
def download_candles(security):
    interval = 10
    start_date = "2010-01-01"
    end_date = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
    url = f"https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities/{security}/candles.json?interval={interval}&from={start_date}&till={end_date}"
    response = requests.get(url)
    result = json.loads(response.text)
    resp_date = result["candles"]["data"]
    columns = result["candles"]["columns"]
    data_shares = pd.DataFrame(resp_date, columns=columns)
    len_df = len(resp_date)
    print(f'Длина первого массива {len_df}')

    last_received_date = data_shares.iloc[-1]["begin"] # код для хранения значения последней даты и времени

    while len_df != 0:
        start_date = (pd.to_datetime(last_received_date) + pd.Timedelta(minutes=interval)).strftime("%Y-%m-%d %H:%M:%S")
        url = f"https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities/{security}/candles.json?interval={interval}&from={start_date}&till={end_date}"
        response = requests.get(url)
        result = json.loads(response.text)
        resp_date = result["candles"]["data"]
        len_df = len(resp_date)
        if len_df == 0:
            break
        data_next_page = pd.DataFrame(resp_date, columns=columns)
        data_shares = pd.concat([data_shares, data_next_page], ignore_index=True)
        last_received_date = data_shares.iloc[-1]["begin"]

    csv_file = data_shares.to_csv(f'raw_data_{security}.csv')
    return csv_file

# Пример использования скачивания данных для акции Газпром
download_candles("GAZP")

Длина первого массива 500
       open   close    high     low        value   volume  \
995  152.51  152.57  152.73  152.38   60432255.2   396220   
996  152.57  153.33  153.40  152.54  346142187.8  2261460   
997  153.36  153.11  153.38  152.86  209132994.9  1366010   
998  153.15  152.80  153.19  152.76   41587331.0   271910   
999  152.85  152.86  153.09  152.68   76243387.8   498830   

                   begin                  end  
995  2014-07-08 12:10:00  2014-07-08 12:19:59  
996  2014-07-08 12:20:00  2014-07-08 12:29:59  
997  2014-07-08 12:30:00  2014-07-08 12:39:59  
998  2014-07-08 12:40:00  2014-07-08 12:49:59  
999  2014-07-08 12:50:00  2014-07-08 12:59:59  
        open   close    high     low        value   volume  \
1495  136.10  136.27  136.33  135.84  148927444.4  1093800   
1496  136.27  136.62  136.67  135.92  125680540.0   922010   
1497  136.57  136.13  136.61  136.13   94737111.1   694750   
1498  136.13  136.43  136.50  136.12   49867620.6   365840   
1499  136

In [68]:
df = pd.read_csv('raw_data_GAZP.csv', index_col=0)
df.head()

,open,close,high,low,value,volume,begin,end
0,144.02,144.16,144.78,144.00,115259065.9,798230,2014-06-09 10:00:00,2014-06-09 10:09:59
1,144.18,143.62,144.20,143.60,92635541.5,643790,2014-06-09 10:10:00,2014-06-09 10:19:59
2,143.62,143.64,143.74,143.50,78514596.6,546760,2014-06-09 10:20:00,2014-06-09 10:29:59
3,143.64,143.68,143.81,143.54,86464723.9,601940,2014-06-09 10:30:00,2014-06-09 10:39:59
4,143.68,143.29,143.72,143.03,116152165.2,810310,2014-06-09 10:40:00,2014-06-09 10:49:59


In [61]:
df[df.duplicated()]

,open,close,high,low,value,volume,begin,end


In [76]:
securities = ['GAZP', 'SBER', 'GMKN', 'LKOH', 'ROSN']
def download_candles():
    for security in securities:
        interval = 10
        start_date = "2010-01-01"
        end_date = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
        url = f"https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities/{security}/candles.json?interval={interval}&from={start_date}&till={end_date}"
        response = requests.get(url, timeout=30)
        result = json.loads(response.text)
        resp_date = result["candles"]["data"]
        columns = result["candles"]["columns"]
        data_shares = pd.DataFrame(resp_date, columns=columns)
        len_df = len(resp_date)
        last_received_date = data_shares.iloc[-1]["begin"] # код для хранения значения последней даты и времени

        while len_df != 0:
            start_date = (pd.to_datetime(last_received_date) + pd.Timedelta(minutes=interval)).strftime("%Y-%m-%d %H:%M:%S")
            url = f"https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities/{security}/candles.json?interval={interval}&from={start_date}&till={end_date}"
            response = requests.get(url, timeout=30)
            result = json.loads(response.text)
            resp_date = result["candles"]["data"]
            len_df = len(resp_date)
            print(len_df)
            if len_df == 0:
                break
            data_next_page = pd.DataFrame(resp_date, columns=columns)
            data_shares = pd.concat([data_shares, data_next_page], ignore_index=True) # Объединяем данные
            last_received_date = data_shares.iloc[-1]["begin"] # записываем последнее значение даты

        data_shares.to_csv(f'raw_data_{security}.csv')

# Запускаем функцию
download_candles()

500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500


In [78]:
def downl_raw_last_day():
    for security in securities:
        interval = 10
        start_date = end_date = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
        url = f"https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities/{security}/candles.json?interval={interval}&from={start_date}&till={end_date}"
        response = requests.get(url)
        result = json.loads(response.text)
        resp_date = result["candles"]["data"]
        columns = result["candles"]["columns"]
        data_shares = pd.DataFrame(resp_date, columns=columns)
        data_shares.to_csv(f'last_day_{security}.csv')

downl_raw_last_day()